In [1]:
import os
import glob

import pandas as pd
import wfdb as wf
import numpy as np
from scipy import signal
import utils.mitdbread as dm
from biosppy.signals import ecg  # R peaks detection algorithm 1
from utils import *  # R peaks detection algorithm 2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
records = dm.get_records()
print(records)

['data/mitdb\\100', 'data/mitdb\\101', 'data/mitdb\\102', 'data/mitdb\\103', 'data/mitdb\\104', 'data/mitdb\\105', 'data/mitdb\\106', 'data/mitdb\\107', 'data/mitdb\\108', 'data/mitdb\\109', 'data/mitdb\\111', 'data/mitdb\\112', 'data/mitdb\\113', 'data/mitdb\\114', 'data/mitdb\\115', 'data/mitdb\\116', 'data/mitdb\\117', 'data/mitdb\\118', 'data/mitdb\\119', 'data/mitdb\\121', 'data/mitdb\\122', 'data/mitdb\\123', 'data/mitdb\\124', 'data/mitdb\\200', 'data/mitdb\\201', 'data/mitdb\\202', 'data/mitdb\\203', 'data/mitdb\\205', 'data/mitdb\\207', 'data/mitdb\\208', 'data/mitdb\\209', 'data/mitdb\\210', 'data/mitdb\\212', 'data/mitdb\\213', 'data/mitdb\\214', 'data/mitdb\\215', 'data/mitdb\\217', 'data/mitdb\\219', 'data/mitdb\\220', 'data/mitdb\\221', 'data/mitdb\\222', 'data/mitdb\\223', 'data/mitdb\\228', 'data/mitdb\\230', 'data/mitdb\\231', 'data/mitdb\\232', 'data/mitdb\\233', 'data/mitdb\\234']


In [4]:
data = np.loadtxt('data/extracted_data/mitdb/100_MLII.csv', delimiter=',')
print(data.shape)

(2270, 188)


In [9]:
import xgboost as xgb
import time
from custom_threading import CustomThread as Thread


def predict(beat_array):
    test_beat = np.array(beat_array)[np.newaxis]
    test_beat = np.transpose(test_beat).T
    result = model.predict(test_beat)
    return result


model = xgb.XGBClassifier()
model.load_model('xgb_ecg.model')

beat0 = data[0][:-1]
beat1 = data[1][:-1]
beat2 = data[2][:-1]
beat3 = data[3][:-1]

start = time.time()

t0 = Thread(target=predict, args=(beat0,))
t1 = Thread(target=predict, args=(beat1,))
t2 = Thread(target=predict, args=(beat2,))
t3 = Thread(target=predict, args=(beat3,))

t0.start()
t1.start()
t2.start()
t3.start()

t0.join()
t1.join()
t2.join()
t3.join()

end = time.time()

print(t0._return)
print(end-start)

[0.]
0.01499176025390625


In [6]:
import xgboost as xgb
import time
import threading

model = xgb.XGBClassifier()
model.load_model('xgb_ecg.model')

start = time.time()

for i in [0, 1, 2, 3]:
    test_beat = np.array(data[i][:-1])[np.newaxis]
    test_beat = np.transpose(test_beat).T
    result = model.predict(test_beat)


end = time.time()

print(end-start)

0.013990402221679688
